# Pulling ERCOT Price node data and train BESS financial model

Using Grid status price node data from ERCOT. You will need an [API key](https://www.gridstatus.io/settings/api) from [Grid Status](https://www.gridstatus.io/)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1466]:
from datetime import datetime

In [2]:
from gridstatusio import GridStatusClient

In [3]:
client = GridStatusClient(api_key="")

In [4]:
client.list_datasets(filter_term="ERCOT LMP")

+-------------------------+-------------------------------------------------------------------------------+
|           Key           |                                     Value                                     |
+-------------------------+-------------------------------------------------------------------------------+
|          Name           |                               ERCOT LMP By Bus                                |
|           ID            |                               ercot_lmp_by_bus                                |
|       Description       |        The Locational Marginal Price for each Electrical Bus in ERCOT,        |
|                         | normally produced by SCED every five minutes. See list of available locations |
|                         |                  by using the ercot_electrical_buses dataset                  |
| Earliest available time |                           2020-01-01T06:00:26+00:00                           |
|  Latest available time  | 

## Pull LMP Data from ERCOT.

Put price node (substation) name in the filter_value parameter

In [1465]:
## Pull LMP Data from ERCOT.

# put station name in the filter_value parameter

data_pnode = client.get_dataset(
    dataset="ercot_lmp_by_bus",
    start="2022-01-01",
    end="2024-07-01",
    tz="US/Central",
    filter_column="location",
    filter_value="DEALEY1_8X",
)

Fetching Page 1...Done in 38.29 seconds. 
Fetching Page 2...Done in 35.02 seconds. Total time: 73.31s. Avg per page: 36.66s
Fetching Page 3...Done in 33.28 seconds. Total time: 106.59s. Avg per page: 35.53s
Fetching Page 4...Done in 35.11 seconds. Total time: 141.7s. Avg per page: 35.43s
Fetching Page 5...Done in 32.97 seconds. Total time: 174.67s. Avg per page: 34.93s
Fetching Page 6...Done in 12.68 seconds. Total time: 187.35s. Avg per page: 31.23s

Total number of rows: 266448


In [ ]:
## use this if you need to pull multiple nodes, as nodes sometimes change name

#data_node = client.get_dataset(
#    dataset="ercot_lmp_by_bus",
#    start="2022-01-01",
#    end="2023-08-01",
#    tz="US/Central",
#    filter_column="location",
#    filter_value="L_CUERO_8_1Y",
#)

#data_node

In [79]:
data_pnode.head(20)

NameError: name 'data_pnode' is not defined

In [1468]:
data_pnode.dtypes

sced_timestamp_local    datetime64[ns, US/Central]
market                                      object
location                                    object
location_type                               object
lmp                                        float64
dtype: object

In [1469]:
# Prep Data 

data_pnode['Delivery.Date'] = pd.to_datetime(data_pnode['sced_timestamp_local'])
data_pnode['Year'] = pd.DatetimeIndex(data_pnode['Delivery.Date']).year
data_pnode['Month'] = pd.DatetimeIndex(data_pnode['Delivery.Date']).month

In [403]:
# prep data if you need to combine data sets

#data_node['Delivery.Date'] = pd.to_datetime(data_node['sced_timestamp_local'])
#data_node['Month'] = pd.DatetimeIndex(data_node['Delivery.Date']).month
#data_node['Year'] = pd.DatetimeIndex(data_node['Delivery.Date']).year
#data1 = data_pnode[data_pnode.Month != 7]
#datafull = data_node.append(data1, ignore_index=True)

In [1470]:
data_pnode = data_pnode.rename(columns={"lmp" : "RTPrice"})

In [1395]:
# Transform data set into 15 minute intervals - if you need to combine with Anc services data or other pricing data

# datafull['count'] = 0
# datafull['interval'] = 0
# interval = 1
# count = 1
# for i in range(len(datafull)):
#     datafull.loc[i,'count'] = count
#     datafull.loc[i,'interval'] = interval
#     interval = interval + 1
#     if interval == 4:
#         interval = 1
#         count = count + 1  

# datafull['RTPrice15'] =  datafull.groupby(['count'])['RTPrice'].transform('mean')
# data15 = datafull[datafull.interval == 1]
# data15 = data15.reset_index(drop=True)
# data15['RTPrice'] = data15['RTPrice15']

# data15.head()

In [1396]:
# Copy the raw data into what will work with

data = data_pnode

In [80]:
# You can also pull data from file

data = pd.read_csv('/Users/nikgupta/Documents/storage/Mergedata20222024.csv')
data = data.rename(columns={"Settlement.Point.Price" : "RTPrice"})
data['Year'] = pd.DatetimeIndex(data['Delivery.Date']).year
data['Month'] = pd.DatetimeIndex(data['Delivery.Date']).month

In [70]:
#data.to_csv('/Users/nikgupta/Documents/storage/datanode_dallas.csv')

In [71]:
## specify your capacity, and the proration that you will give to energy vs ancillary services
# if you are running the program with 15 minute data, replace 12 with 4

MhW = 18
Capacity = 9
ProRation = .5
MaxCharge = MhW * ProRation
ChargeRate = ( Capacity * (1 - ProRation ) ) / 4

In [72]:
print(ChargeRate)

1.125


In [73]:
# variables needed for training

data['Cycle'] = 0
data['Charge'] = 0
data['Revenue'] = 0.0
data['Available'] = 0.0
data['TestAvailable'] = 0.0
data['TestCharge'] = 0.0

In [125]:
data.head()

,Unnamed: 0,Delivery.Date,Delivery.Hour,Delivery.Interval,Settlement.Point.Name,Settlement.Point.Type,RTPrice,REGDN,REGUP,RRS,...,Revenue,Available,TestDischargeRevenue,TestChargeRevenue,TestDischarge,TestCharge,CostBuy,CostSell,BuyPrice,SellPrice
0,1,01/01/2022,1,1,HB_SOUTH,HU,48.97,NaN,NaN,NaN,...,0.0,0.0,0.00000,0,9.000,7.875,-22.97,22.97,0.0,0.0
1,2,01/01/2022,1,2,HB_SOUTH,HU,106.25,NaN,NaN,NaN,...,0.0,0.0,119.53125,0,7.875,6.750,-85.71,85.71,0.0,0.0
2,3,01/01/2022,1,3,HB_SOUTH,HU,37.89,NaN,NaN,NaN,...,0.0,0.0,42.62625,0,6.750,5.625,-17.85,18.53,0.0,0.0
3,4,01/01/2022,1,4,HB_SOUTH,HU,32.60,NaN,NaN,NaN,...,0.0,0.0,36.67500,0,5.625,4.500,-13.24,14.29,0.0,0.0
4,5,01/01/2022,2,1,HB_SOUTH,HU,29.64,NaN,NaN,NaN,...,0.0,0.0,33.34500,0,4.500,3.375,-11.33,7.22,0.0,0.0


## Finding Optimal Dispatch

At each time point, the battery needs to decide whether to charge or discharge. This program sets strike prices that optimize the batteries decision making, the strike price is the benefit or cost that the battery takes by charging or discharging. After finding the strike prices, the bettery will buy if the real time price is below the strike price for charging, and or sell if the real time price is above the strike price for discharging

The program below will 
1. Use Monthly aggregates as the initial strike prices and do a quick run to demonstrate the initial baseline
2. Calculate the strike prices through backward induction
3. Calculate the result of close to optimal charging

There are two backward induction programs. The first is more accuate but runs more slowly, while the second one runs faster but provides lower revenue.


In [81]:
# Set the monthly average as the initial strike prices

data['StrikeBuy'] =  data.groupby(['Month','Year'])['RTPrice'].transform('mean')
data['StrikeSell'] =  data.groupby(['Month','Year'])['RTPrice'].transform('mean')

In [84]:
## Run an initial run of charge and discharge as a benchmark
# charge if prices are below strike price and sell if the price is above the strike price

data['Cycle'] = 0
data['Charge'] = 0
data['Revenue'] = 0.0
data['Available'] = 0.0

for i in range(len(data)):
        if i != 0:
            data.loc[i,'Available'] = data.loc[i-1,'Available'] 
        else:
            data.loc[i,'Available'] = 0
        
        if data.loc[i,'RTPrice'] < data.loc[i,'StrikeBuy'] and data.loc[i,'Available'] < MaxCharge:
            data.loc[i,'Available'] = data.loc[i,'Available'] + ChargeRate
            data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * -1
            data.loc[i,'Charge'] = 1
    
        if data.loc[i,'RTPrice'] > data.loc[i,'StrikeSell'] and data.loc[i,'Available'] > 0 and data.loc[i,'Charge'] != 1:
            data.loc[i,'Available'] = data.loc[i,'Available'] - ChargeRate
            data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice']
            data.loc[i,'Cycle'] = 1


In [85]:
data['Revenue'].sum()

604049.8274999999

In [126]:
# Slow version
## Runs slower but has higher revenue

data['TestDischargeRevenue'] = 0.0
data['TestChargeRevenue'] = 0.0
data['TestDischarge'] = MaxCharge
data['TestCharge'] = 0.0
data['Available'] = 0.0


for i in reversed(range(len(data))):
    ## counter
    if i % 5000 == 0:
        print(i)
    
    # test charge
    ## Baseline
    discharged = 0
    data['TestCharge'] = MaxCharge - ChargeRate
    
    for j in range(i+1,len(data)):
        data.loc[j,'TestCharge'] = data.loc[j-1,'TestCharge'] 
        
        if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'TestCharge'] > 0:
            data.loc[j,'TestCharge'] = data.loc[j,'TestCharge'] - ChargeRate
            data.loc[j,'TestChargeRevenue'] = ChargeRate * data.loc[j,'RTPrice']
            for j in range(j+1,len(data)):
                data.loc[j,'TestCharge'] = data.loc[j-1,'TestCharge']
                if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'TestCharge'] > 0:
                    data.loc[j,'TestCharge'] = data.loc[j,'TestCharge'] - ChargeRate
                    data.loc[j,'TestChargeRevenue'] = ChargeRate * data.loc[j,'RTPrice']
                if data.loc[j,'TestCharge'] <= 0.0:
                    discharged = 1
                    break

        elif data.loc[j,'TestCharge'] < MaxCharge:
            data.loc[j,'TestCharge'] = data.loc[j,'TestCharge'] + ChargeRate
            data.loc[j,'TestChargeRevenue'] = ChargeRate * data.loc[j,'RTPrice'] * -1.0
        
        if discharged == 1:
            break
    
    NoChargeCase = data['TestChargeRevenue'].sum()

    #reset 
    data['Revenue'] = 0
    
    ## test charge
    data['Available'] = 0.0
    data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * -1.0 
    data.loc[i,'Available'] = MaxCharge
    
    discharged = 0 
    for j in range(i+1,len(data)):
        data.loc[j,'Available'] = data.loc[j-1,'Available'] 
            
        if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'Available'] > 0:
            data.loc[j,'Available'] = data.loc[j,'Available'] - ChargeRate
            data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice']
            for j in range(j+1,len(data)):
                data.loc[j,'Available'] = data.loc[j-1,'Available'] 
                if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'Available'] > 0:
                    data.loc[j,'Available'] = data.loc[j,'Available'] - ChargeRate
                    data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice']
                if data.loc[j,'Available'] <= 0.0:
                    discharged = 1
                    break

        if discharged == 1:
            break
    
    ChargeCase = data['Revenue'].sum()

    data.loc[i,'CostBuy'] = ChargeCase - NoChargeCase 
    data.loc[i,'CostBuy'] =  data.loc[i,'CostBuy']/ ChargeRate
    data.loc[i, 'StrikeBuy'] = data.loc[i,'RTPrice'] + data.loc[i,'CostBuy']

    data['TestChargeRevenue'] = 0

    ## test no discharge
    for j in range(i+1,len(data)):
        data.loc[j,'TestDischarge'] = data.loc[j-1,'TestDischarge']
        if data.loc[j,'RTPrice'] <= data.loc[j,'StrikeBuy'] and data.loc[j,'TestDischarge'] < MaxCharge:
            data.loc[j,'TestDischarge'] = data.loc[j,'TestDischarge'] + ChargeRate
            data.loc[j,'TestDischargeRevenue'] = ChargeRate * data.loc[j,'RTPrice'] * -1.0   
        elif data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'TestDischarge'] > 0:
            data.loc[j,'TestDischarge'] = data.loc[j,'TestDischarge'] - ChargeRate
            data.loc[j,'TestDischargeRevenue'] = ChargeRate * data.loc[j,'RTPrice']
        else:
            data.loc[j,'TestDischargeRevenue'] = 0
        
        if data.loc[j,'TestDischarge'] == 0.0:
            break
   
    NoDischargeCase = data['TestDischargeRevenue'].sum()

    # Reset
    data['Available'] = 0.0
    data['Revenue'] = data['TestDischargeRevenue']
    
    ## test discharge
    data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * 1.0
    data.loc[i,'Available'] = MaxCharge - ChargeRate

    for j in range(i+1,len(data)):
        data.loc[j,'Available'] = data.loc[j-1,'Available']
        if data.loc[j,'RTPrice'] <= data.loc[j,'StrikeBuy'] and data.loc[j,'Available'] < MaxCharge:
            data.loc[j,'Available'] = data.loc[j,'Available'] + ChargeRate
            data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice'] * -1.0   
        elif data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'Available'] > 0:
            data.loc[j,'Available'] = data.loc[j,'Available'] - ChargeRate
            data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice']
        else:
            data.loc[j,'Revenue'] = 0
    
        if data.loc[j,'Available'] == data.loc[j,'TestDischarge']:
            break
    
    DischargeCase = data['Revenue'].sum()
    
    data.loc[i,'CostSell'] = DischargeCase - NoDischargeCase 
    data.loc[i,'CostSell'] =  data.loc[i,'CostSell']/ ChargeRate
    data.loc[i, 'StrikeSell'] = data.loc[i,'RTPrice'] - data.loc[i,'CostSell']


120000
115000
110000
105000
100000
95000
90000
85000
80000
75000
70000
65000
60000
55000
50000
45000
40000
35000
30000
25000
20000
15000
10000
5000
0


In [ ]:
# Fast version 
## Faster Training time but lower revenue

data['TestDischargeRevenue'] = 0.0
data['TestChargeRevenue'] = 0.0
data['TestDischarge'] = MaxCharge
data['TestCharge'] = 0.0
data['Available'] = 0.0


for i in reversed(range(len(data))):
    ## counter
    if i % 5000 == 0:
        print(i)
    
    # test charge
    ## Baseline
    discharged = 0
    data['TestCharge'] = MaxCharge - ChargeRate
    
    for j in range(i+1,len(data)):
        data.loc[j,'TestCharge'] = data.loc[j-1,'TestCharge'] 
        
        if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'TestCharge'] > 0:
            data.loc[j,'TestCharge'] = data.loc[j,'TestCharge'] - ChargeRate
            data.loc[j,'TestChargeRevenue'] = ChargeRate * data.loc[j,'RTPrice']
            for j in range(j+1,len(data)):
                data.loc[j,'TestCharge'] = data.loc[j-1,'TestCharge']
                if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'TestCharge'] > 0:
                    data.loc[j,'TestCharge'] = data.loc[j,'TestCharge'] - ChargeRate
                    data.loc[j,'TestChargeRevenue'] = ChargeRate * data.loc[j,'RTPrice']
                if data.loc[j,'TestCharge'] <= 0.0:
                    discharged = 1
                    break

        elif data.loc[j,'TestCharge'] < MaxCharge:
            data.loc[j,'TestCharge'] = data.loc[j,'TestCharge'] + ChargeRate
            data.loc[j,'TestChargeRevenue'] = ChargeRate * data.loc[j,'RTPrice'] * -1.0      
            
        if discharged == 1:
            break
    
    NoChargeCase = data['TestChargeRevenue'].sum()

    #reset 
    data['Revenue'] = 0
    
    ## test charge
    data['Available'] = 0.0
    data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * -1.0 
    data.loc[i,'Available'] = MaxCharge
    
    discharged = 0 
    for j in range(i+1,len(data)):
        data.loc[j,'Available'] = data.loc[j-1,'Available'] 
            
        if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'Available'] > 0:
            data.loc[j,'Available'] = data.loc[j,'Available'] - ChargeRate
            data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice']
            for j in range(j+1,len(data)):
                data.loc[j,'Available'] = data.loc[j-1,'Available'] 
                if data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'Available'] > 0:
                    data.loc[j,'Available'] = data.loc[j,'Available'] - ChargeRate
                    data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice']
                if data.loc[j,'Available'] <= 0.0:
                    discharged = 1
                    break

        if discharged == 1:
            break
    
    ChargeCase = data['Revenue'].sum()

    data.loc[i,'CostBuy'] = ChargeCase - NoChargeCase 
    data.loc[i,'CostBuy'] =  data.loc[i,'CostBuy']/ ChargeRate
    data.loc[i, 'StrikeBuy'] = data.loc[i,'RTPrice'] + data.loc[i,'CostBuy']

    data['TestChargeRevenue'] = 0

    ## test no discharge
    for j in range(i+1,len(data)):
        pretrain = data.loc[j-1,'TestDischarge']
        data.loc[j,'TestDischarge'] = data.loc[j-1,'TestDischarge']
        if data.loc[j,'RTPrice'] <= data.loc[j,'StrikeBuy'] and data.loc[j,'TestDischarge'] < MaxCharge:
            data.loc[j,'TestDischarge'] = data.loc[j,'TestDischarge'] + ChargeRate
            data.loc[j,'TestDischargeRevenue'] = ChargeRate * data.loc[j,'RTPrice'] * -1.0   
        elif data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'TestDischarge'] > 0:
            data.loc[j,'TestDischarge'] = data.loc[j,'TestDischarge'] - ChargeRate
            data.loc[j,'TestDischargeRevenue'] = ChargeRate * data.loc[j,'RTPrice']
        else:
            data.loc[j,'TestDischargeRevenue'] = 0
        
        if data.loc[j,'TestDischarge'] == pretrain:
            break
   
    NoDischargeCase = data['TestDischargeRevenue'].sum()

    # Reset
    data['Available'] = 0.0
    data['Revenue'] = data['TestDischargeRevenue']
    
    ## test discharge
    data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * 1.0
    data.loc[i,'Available'] = MaxCharge - ChargeRate

    for j in range(i+1,len(data)):
        data.loc[j,'Available'] = data.loc[j-1,'Available']
        if data.loc[j,'RTPrice'] <= data.loc[j,'StrikeBuy'] and data.loc[j,'Available'] < MaxCharge:
            data.loc[j,'Available'] = data.loc[j,'Available'] + ChargeRate
            data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice'] * -1.0   
        elif data.loc[j,'RTPrice'] > data.loc[j,'StrikeSell'] and data.loc[j,'Available'] > 0:
            data.loc[j,'Available'] = data.loc[j,'Available'] - ChargeRate
            data.loc[j,'Revenue'] = ChargeRate * data.loc[j,'RTPrice']
        else:
            data.loc[j,'Revenue'] = 0
    
        if data.loc[j,'Available'] == data.loc[j,'TestDischarge']:
            break
    
    DischargeCase = data['Revenue'].sum()
    
    data.loc[i,'CostSell'] = DischargeCase - NoDischargeCase 
    data.loc[i,'CostSell'] =  data.loc[i,'CostSell']/ ChargeRate
    data.loc[i, 'StrikeSell'] = data.loc[i,'RTPrice'] - data.loc[i,'CostSell']


In [127]:
## run charge and discharge again now that strike prices have been optimized

data['Cycle'] = 0
data['Charge'] = 0
data['Revenue'] = 0.0
data['Available'] = 0.0

for i in range(len(data)):
        if i != 0:
            data.loc[i,'Available'] = data.loc[i-1,'Available'] 
        else:
            data.loc[i,'Available'] = 0
        
        if data.loc[i,'RTPrice'] < data.loc[i,'StrikeBuy'] and data.loc[i,'Available'] < MaxCharge:
            data.loc[i,'Available'] = data.loc[i,'Available'] + ChargeRate
            data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * -1
            data.loc[i,'Charge'] = 1
    
        if data.loc[i,'RTPrice'] > data.loc[i,'StrikeSell'] and data.loc[i,'Available'] > 0 and data.loc[i,'Charge'] != 1:
            data.loc[i,'Available'] = data.loc[i,'Available'] - ChargeRate
            data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice']
            data.loc[i,'Cycle'] = 1


In [128]:
data['Revenue'].sum()

1740249.63

In [ ]:
data = pd.read_csv('/Users/nikgupta/Documents/storage/datanode.csv')

## Study Cycle rate

set threshold to site a price for cycles. If the difference between the real time price strike price is not above the threshold, ie the cost of cycles, the battery will not discharge. 


In [200]:
## set threshold to site a price for cycles, ie a price where you won't sell if the RT price is above 
## strike price in the 5 minute interval do dampen the number of cycles

data['Charge'] = 0
data['Cycle'] = 0
data['Available'] = 0.0
data['Revenue'] = 0.0
Threshold = 90

for i in range(len(data)):
        if i != 0:
            data.loc[i,'Available'] = data.loc[i-1,'Available'] 
        else:
            data.loc[i,'Available'] = 0
        if data.loc[i,'RTPrice'] < data.loc[i,'StrikeBuy'] and data.loc[i,'Available'] < MaxCharge:
            data.loc[i,'Available'] = data.loc[i,'Available'] + ChargeRate
            data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice'] * -1
            data.loc[i,'Charge'] = 1 
            
        if data.loc[i,'RTPrice'] > data.loc[i,'StrikeSell'] and data.loc[i,'Available'] > 0 and data.loc[i,'Charge'] != 1:           
            if ChargeRate * (data.loc[i,'RTPrice'] - data.loc[i,'StrikeSell']) >= Threshold:
                data.loc[i,'Available'] = data.loc[i,'Available'] - ChargeRate
                data.loc[i,'Revenue'] = ChargeRate * data.loc[i,'RTPrice']
                data.loc[i,'Cycle'] = 1

In [201]:
data['Revenue'].sum()

1331127.91125

In [204]:
table = data.pivot_table(index=['Year','Month'], values=['Revenue','Available','Cycle'],
                         aggfunc={'Revenue':"sum",'Available':"sum",'Cycle':"sum"})

In [205]:
table

Available  Cycle       Revenue
Year Month                                
2022 1      26347.500     19    3973.06125
     2      23582.250     34   38593.76625
     3      25903.125     55   14339.12625
     4      24336.000     66   33212.62125
     5      24372.000    125   70651.37250
     6      25785.000      9    1488.89250
     7      23627.250    119  121235.67000
     8      24765.750    107   31166.91000
     9      25128.000     34    7859.13750
     10     25590.375     46   18818.02125
     11     25479.000     29    7456.38750
     12     26163.000     42   39316.12875
2023 1      53422.875     14    3914.49375
     2      47953.125     26    4171.03875
     3      51858.000     55   16151.88375
     4      51021.000     32    7406.07750
     5      51693.750     44   19145.93625
     6      49429.125     81   64175.33250
     7      51449.625     60   48807.49500
     8      42027.750    353  464125.41000
     9      47778.750    128  168087.38625
     10     51487.875     69   26417.82375
     11     50447.250     60   16052.71500
     12     52849.125     26    8690.04000
2024 1      26006.625     42   12506.16375
     2      24973.875      7    1682.20125
     3      26106.750     34    9242.87625
     4      24581.250     61   28291.11750
     5      25609.500     58   38835.47250
     6      25349.625     26    5313.35250

In [206]:
table.to_csv('/Users/nikgupta/Documents/storage/DischargeSummary_200.csv')

In [178]:
anctable.to_csv('/Users/nikgupta/Documents/storage/anctable.csv')

NameError: name 'anctable' is not defined

In [131]:
data.to_csv('/Users/nikgupta/Documents/storage/datatest.csv')

## Find revenue by price tier

How much revenue comes from price peaks? The code below finds how much revenue comes from different at different real time prices.

In [47]:
Tier1 = data.loc[data['RTPrice'] < 1000].groupby(['Month'])['Revenue'].sum()
Tier1000 = data.loc[(data['RTPrice'] > 1000) & (data['RTPrice'] <= 2000)].groupby(['Month'])['Revenue'].sum()
Tier2000 = data.loc[(data['RTPrice'] > 2000) & (data['RTPrice'] <= 3000)].groupby(['Month'])['Revenue'].sum()
Tier3000 = data.loc[(data['RTPrice'] > 3000) & (data['RTPrice'] <= 4000)].groupby(['Month'])['Revenue'].sum()
Tier4000 = data.loc[data['RTPrice'] > 4000].groupby(['Month'])['Revenue'].sum()

In [48]:
Tier1 = Tier1.reset_index()
Tier1000 = Tier1000.reset_index()
Tier2000 = Tier2000.reset_index()
Tier3000 = Tier3000.reset_index()
Tier4000 = Tier4000.reset_index()

In [49]:
Tier1 = Tier1.rename(columns={"Revenue" : "LT1000"})
Tier1000 = Tier1000.rename(columns={"Revenue" : "GT1000"})
Tier2000 = Tier2000.rename(columns={"Revenue" : "GT2000"})
Tier3000 = Tier3000.rename(columns={"Revenue" : "GT3000"})
Tier4000 = Tier4000.rename(columns={"Revenue" : "GT4000"})

In [50]:
PriceTier = pd.merge(Tier1, Tier1000, on='Month', how='left')
PriceTier = pd.merge(PriceTier, Tier2000, on='Month', how='left')
PriceTier = pd.merge(PriceTier, Tier3000, on='Month', how='left')
PriceTier = pd.merge(PriceTier, Tier4000, on='Month', how='left')

In [51]:
PriceTier

,Month,LT1000,GT1000,GT2000,GT3000,GT4000
0,1,67251.9125,1546.2000,2897.5375,NaN,1859.6500
1,2,53762.1875,30134.8125,16519.6375,-8417.9250,150989.4625
2,3,90765.5125,4154.9375,NaN,NaN,NaN
3,4,87995.1000,14301.7250,2501.2500,4459.9750,NaN
4,5,97841.2000,10187.4000,5491.5000,5002.7750,5725.3875
5,6,65143.3875,10900.4500,6438.1625,8758.5375,23400.4625
6,7,96824.3000,33602.3125,4909.3125,4470.1250,47356.6875
7,8,143446.3750,71308.2000,41851.3000,31655.0000,131648.8375
8,9,94936.0875,23554.9875,12336.5125,21553.4750,26848.5375
9,10,99215.9125,18867.4625,5597.3750,NaN,5780.7000


In [34]:
PriceTier.to_csv('/Users/nikgupta/Documents/PriceTiersSummary.csv')